 # Analyse Exploratoire (EDA) - Posts Crypto (Reddit & Twitter)

 **Objectif:** Analyser un dataset consolidé de posts Reddit et Twitter pour comprendre les tendances, la qualité des données et les sujets de discussion.

 **Fichier:** `consolidated_data.csv`
 **Colonnes:** `id`, `text`, `date`, `source`, `author`, `engagement`, `crypto_mentioned`, `date_only`



 ## 0. Configuration et Import des Bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

# Imports pour l'analyse de texte
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords

# Imports pour l'analyse de qualité
import emoji
from langdetect import detect, LangDetectException

# Configuration des graphiques
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("Bibliothèques importées avec succès.")

## 1. Chargement et Nettoyage Initial des Données
Chargement du CSV et conversion des types de données (notamment les dates).

In [ ]:
# Chargement des données
try:
    df = pd.read_csv(r"D:\CryptoVibe\CryptoVibe\data\Bronze\consolidated_data.csv")
    print(f"Données chargées avec succès: {df.shape[0]} lignes et {df.shape[1]} colonnes.")
except FileNotFoundError:
    print("Erreur: Le fichier 'consolidated_data.csv' n'a pas été trouvé.")
    # Création d'un DataFrame de test si le fichier n'existe pas
    data = {
        'id': range(1000),
        'text': ['Bitcoin is up!', 'I love #ETH', 'What about $SOL ?', 'This is a test post', 'http://spam.com', 'BTC BTC BTC', '🚀🌕', 'DOGE to the moon', 'Buy $SHIB now!', 'This is a very long post designed to test the limits of what a post can be, potentially exceeding the 1000 character limit just for fun.'] * 100,
        'date': pd.to_datetime(pd.date_range(start='2023-01-01', periods=1000, freq='H')),
        'source': ['twitter', 'reddit'] * 500,
        'author': ['user_a', 'user_b', 'user_c', 'user_d', 'user_e'] * 200,
        'engagement': np.random.randint(0, 5000, 1000),
        'crypto_mentioned': [np.nan] * 1000,
        'date_only': pd.to_datetime(pd.date_range(start='2023-01-01', periods=1000, freq='H')).date
    }
    df = pd.DataFrame(data)
    print("Un jeu de données de test a été créé.")

# Affichage des premières lignes et des informations
print("\n--- Aperçu des données ---")
print(df.head())

print("\n--- Informations sur le DataFrame ---")
df.info()

In [ ]:
# --- Nettoyage Initial et Feature Engineering ---

# Conversion de la date (essentiel)
df['date'] = pd.to_datetime(df['date'])
df['date_only'] = pd.to_datetime(df['date_only'])

# Extraction des composantes temporelles
df['hour'] = df['date'].dt.hour
df['day_of_week'] = df['date'].dt.day_name()
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Nettoyage du texte (remplir les NaNs)
df['text'] = df['text'].fillna('')

# Calcul de la longueur du texte
df['text_length'] = df['text'].str.len()

print("\nColonnes temporelles et 'text_length' ajoutées.")
print(df[['date', 'hour', 'day_of_week', 'text_length']].head())

## 2.  Statistiques Générales
Analyse de la taille du dataset, de la distribution des sources, des auteurs et de l'engagement.

In [ ]:
# --- Statistiques Générales ---

print(f"Nombre total de posts: {len(df)}")
print(f"Plage de dates: Du {df['date'].min()} au {df['date'].max()}")
print(f"Nombre d'auteurs uniques: {df['author'].nunique()}")

print("\n--- Statistiques d'Engagement ---")
print(df['engagement'].describe(percentiles=[.25, .5, .75]).to_markdown(floatfmt=".2f"))

In [ ]:
# --- Distribution par Source (Pie Chart) ---

source_counts = df['source'].value_counts()

plt.figure(figsize=(8, 6))
plt.pie(source_counts, labels=source_counts.index, autopct='%1.1f%%', startangle=140, colors=['#1DA1F2', '#FF4500'])
plt.title('Distribution des Posts par Source (Twitter vs Reddit)')
plt.ylabel('')
plt.show()

In [ ]:
# --- Distribution des Posts par Auteur ---

author_posts = df['author'].value_counts()
power_users = author_posts[author_posts > 50] # Seuil défini dans les requirements (section 5)

print(f"\nNombre d'auteurs 'Power Users' (> 50 posts): {len(power_users)}")
print("Top 10 des 'Power Users':")
print(power_users.head(10).to_markdown())

# Visualisation de la distribution (très asymétrique, d'où l'échelle log)
plt.figure(figsize=(12, 6))
sns.histplot(author_posts, bins=100, log_scale=(False, True))
plt.title('Distribution du Nombre de Posts par Auteur')
plt.xlabel('Nombre de Posts')
plt.ylabel("Nombre d'Auteurs (Échelle Log)")
plt.show()

## 3.  Analyse Temporelle
Examen de l'évolution du volume de posts dans le temps.

In [ ]:
# --- Posts par Jour (Line Chart) ---

posts_per_day = df.groupby('date_only').size()

plt.figure(figsize=(15, 7))
posts_per_day.plot(kind='line', label='Posts par Jour')
plt.title('Nombre de Posts par Jour au Fil du Temps')
plt.xlabel('Date')
plt.ylabel('Nombre de Posts')

# --- Tendance Temporelle (Regression Line) ---
posts_per_day_df = posts_per_day.reset_index(name='count')
posts_per_day_df['day_num'] = (posts_per_day_df['date_only'] - posts_per_day_df['date_only'].min()).dt.days

sns.regplot(
    x='day_num',
    y='count',
    data=posts_per_day_df,
    line_kws={'color':'red', 'linestyle': '--'},
    scatter_kws={'alpha':0.3, 's':10},
    label='Tendance'
)
plt.legend()
plt.show()

In [ ]:
# --- Identification des Pics d'Activité ---

# Un "pic" est défini comme un jour dépassant la moyenne + 2 écarts-types
mean_posts = posts_per_day.mean()
std_posts = posts_per_day.std()
activity_threshold = mean_posts + (2 * std_posts)

activity_spikes = posts_per_day[posts_per_day > activity_threshold]

print(f"Seuil de pic d'activité (Moyenne + 2*STD): {activity_threshold:.2f} posts/jour")
print(f"Jours avec une activité de pointe ({len(activity_spikes)} jours):")
print(activity_spikes.sort_values(ascending=False).to_markdown())

In [ ]:
# --- Posts par Heure et Jour de la Semaine ---

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Posts par heure
sns.countplot(ax=axes[0], x=df['hour'], palette="viridis")
axes[0].set_title('Distribution des Posts par Heure de la Journée')
axes[0].set_xlabel('Heure (0-23)')
axes[0].set_ylabel('Nombre de Posts')

# Posts par jour de la semaine
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.countplot(ax=axes[1], x=df['day_of_week'], order=days_order, palette="plasma")
axes[1].set_title('Distribution des Posts par Jour de la Semaine')
axes[1].set_xlabel('Jour de la Semaine')
axes[1].set_ylabel('Nombre de Posts')

plt.tight_layout()
plt.show()

## 4.  Analyse du Texte
Analyse de la longueur des textes, des mots et n-grams les plus fréquents.

In [ ]:
# --- Distribution de la Longueur du Texte ---

print("--- Statistiques de la Longueur du Texte ---")
print(df['text_length'].describe(percentiles=[.25, .5, .75]).to_markdown(floatfmt=".2f"))

plt.figure(figsize=(12, 6))
sns.histplot(df['text_length'], bins=100, kde=True)
plt.title('Distribution de la Longueur du Texte')
plt.xlabel('Longueur du Texte (caractères)')
plt.ylabel('Fréquence')
plt.xlim(0, 1000) # Limite pour une meilleure lisibilité
plt.show()

In [ ]:
# --- Posts Courts et Longs ---

short_posts_count = (df['text_length'] < 10).sum()
long_posts_count = (df['text_length'] > 1000).sum()

print(f"Posts trop courts (< 10 caractères): {short_posts_count} ({short_posts_count / len(df) * 100:.2f}%)")
print(f"Posts trop longs (> 1000 caractères): {long_posts_count} ({long_posts_count / len(df) * 100:.2f}%)")

if short_posts_count > 0:
    print("\nExemples de posts courts:")
    print(df[df['text_length'] < 10]['text'].value_counts().head().to_markdown())




In [ ]:
# --- Préparation pour l'analyse des mots (N-grams) ---

# Téléchargement des stop words (à faire une fois)
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

# Liste des stop words (anglais)
stop_words = set(stopwords.words('english'))
# Ajout de termes crypto courants ou de bruit pour les exclure de l'analyse générale
custom_stopwords = ['crypto', 'bitcoin', 'eth', 'btc', 'https', 'co', 'com', 'www', 't', 's', 'http', 'rt']
stop_words.update(custom_stopwords)

# Fonction pour obtenir les top N-grams
def get_top_ngrams(corpus, ngram_range=(1, 1), n=20):
    vec = CountVectorizer(
        stop_words=list(stop_words),
        ngram_range=ngram_range,
        token_pattern=r'\b[a-zA-Z]{2,}\b' # Mots d'au moins 2 lettres
    ).fit(corpus)
    
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
# --- Top 20 Mots (Unigrams) ---

top_unigrams = get_top_ngrams(df['text'], ngram_range=(1, 1), n=20)
df_top_unigrams = pd.DataFrame(top_unigrams, columns=['Mot', 'Fréquence'])

plt.figure(figsize=(12, 8))
sns.barplot(x='Fréquence', y='Mot', data=df_top_unigrams, palette='coolwarm')
plt.title('Top 20 des Mots les Plus Fréquents (Unigrams)')
plt.show()

In [ ]:
# --- Top 20 Bigrams et Trigrams ---

top_bigrams = get_top_ngrams(df['text'], ngram_range=(2, 2), n=20)
df_top_bigrams = pd.DataFrame(top_bigrams, columns=['Bigram', 'Fréquence'])

top_trigrams = get_top_ngrams(df['text'], ngram_range=(3, 3), n=20)
df_top_trigrams = pd.DataFrame(top_trigrams, columns=['Trigram', 'Fréquence'])

fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Plot Bigrams
sns.barplot(ax=axes[0], x='Fréquence', y='Bigram', data=df_top_bigrams, palette='Greens_d')
axes[0].set_title('Top 20 des Bigrams les Plus Fréquents')

# Plot Trigrams
sns.barplot(ax=axes[1], x='Fréquence', y='Trigram', data=df_top_trigrams, palette='Blues_d')
axes[1].set_title('Top 20 des Trigrams les Plus Fréquents')

plt.tight_layout()
plt.show()

In [ ]:
# --- Word Cloud ---

text_corpus = " ".join(text for text in df['text'])
wordcloud = WordCloud(stopwords=stop_words, background_color="white", width=1000, height=500, max_words=150).generate(text_corpus)

plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Word Cloud du Vocabulaire Complet')
plt.show()

## 5.  Détection Crypto (Préliminaire)
Comptage manuel des mentions de cryptomonnaies populaires.

In [ ]:
# Définition des termes à rechercher (expressions régulières non sensibles à la casse)
crypto_terms_map = {
    'Bitcoin': r'\b(bitcoin|btc|₿)\b',
    'Ethereum': r'\b(ethereum|eth)\b',
    'Solana': r'\b(solana|sol)\b',
    'Binance Coin': r'\b(binance|bnb)\b',
    'Cardano': r'\b(cardano|ada)\b',
    'Ripple': r'\b(ripple|xrp)\b',
    'Dogecoin': r'\b(dogecoin|doge)\b',
    'Shiba Inu': r'\b(shiba|shib)\b',
    'Polygon': r'\b(polygon|matic)\b',
    'Litecoin': r'\b(litecoin|ltc)\b',
}

crypto_counts = {}
df['has_crypto'] = False
text_lower = df['text'].str.lower() # Optimisation: mettre en minuscule une seule fois

for name, pattern in crypto_terms_map.items():
    mentions = text_lower.str.contains(pattern, regex=True, case=False)
    crypto_counts[name] = mentions.sum()
    df['has_crypto'] = df['has_crypto'] | mentions # Marque True si au moins une crypto est trouvée

# Dataframe des comptes
df_crypto_counts = pd.DataFrame.from_dict(crypto_counts, orient='index', columns=['Mentions'])
df_crypto_counts = df_crypto_counts.sort_values('Mentions', ascending=False)

print("--- Mentions des Top Cryptos ---")
print(df_crypto_counts.to_markdown())

In [ ]:
# --- Bar Chart des Top Cryptos ---

plt.figure(figsize=(12, 8))
sns.barplot(x=df_crypto_counts['Mentions'], y=df_crypto_counts.index, palette="rocket")
plt.title('Top 10 des Cryptos Mentionnées')
plt.xlabel('Nombre de Mentions')
plt.ylabel('Cryptomonnaie')
plt.show()

# --- Pourcentage de posts avec mention ---
percent_with_crypto = df['has_crypto'].mean() * 100
print(f"\n{percent_with_crypto:.2f}% des posts mentionnent au moins une crypto de la liste.")

## 6.  Analyse de Qualité et Détection de Spam
Identification des problèmes potentiels dans les données : URLs, emojis, doublons, spam.

In [ ]:
# --- Posts avec URLs ---

df['has_url'] = df['text'].str.contains(r'http\S+|www\.\S+', regex=True)
url_counts = df['has_url'].value_counts()

print(f"Posts avec URL: {url_counts.get(True, 0)}")
print(f"Posts sans URL: {url_counts.get(False, 0)}")

sns.barplot(x=url_counts.index, y=url_counts.values)
plt.title('Proportion de Posts Contenant une URL')
plt.xlabel('Contient une URL')
plt.ylabel('Nombre de Posts')
plt.xticks([0, 1], ['Non', 'Oui'])
plt.show()

In [ ]:
# --- Analyse des Emojis ---

def extract_emojis(text):
    return [char for char in text if emoji.is_emoji(char)]

# Ceci peut être lent sur un gros dataset
# Création d'une liste complète de tous les emojis
try:
    all_emojis_list = df['text'].apply(extract_emojis).sum()
    emoji_counts = Counter(all_emojis_list)
    df_top_emojis = pd.DataFrame(emoji_counts.most_common(10), columns=['Emoji', 'Compte'])
    
    print("--- Top 10 Emojis ---")
    print(df_top_emojis.to_markdown(index=False))

    # Proportion de posts avec emojis
    df['has_emoji'] = df['text'].apply(lambda x: len(extract_emojis(x)) > 0)
    print(f"\nProportion de posts avec au moins 1 emoji: {df['has_emoji'].mean() * 100:.2f}%")

except Exception as e:
    print(f"Erreur lors de l'analyse des emojis: {e}")

In [ ]:
# --- Détection de la Langue ---

# AVERTISSEMENT: C'est TRÈS LENT. Nous utilisons un échantillon.
print("\n--- Détection de la langue (sur un échantillon de 1000 posts) ---")

sample_size = min(1000, len(df))
df_sample = df.sample(sample_size, random_state=42)

def detect_lang_safe(text):
    if not text or not text.strip():
        return 'unknown'
    try:
        return detect(text)
    except LangDetectException:
        return 'error'

df_sample['language'] = df_sample['text'].apply(detect_lang_safe)
lang_distribution = df_sample['language'].value_counts(normalize=True).head(5)

print(lang_distribution.to_markdown(floatfmt=".2%"))

In [ ]:
# --- Doublons et Spam ---

# Duplicates exacts
exact_duplicates = df.duplicated(subset=['text']).sum()
print(f"\nNombre de posts avec texte exact en double: {exact_duplicates} ({exact_duplicates / len(df) * 100:.2f}%)")

# Texte répété (Spam)
repeated_text_counts = df['text'].value_counts()
spammy_text = repeated_text_counts[repeated_text_counts > 5]
print(f"\nNombre de textes uniques répétés plus de 5 fois: {len(spammy_text)}")
if len(spammy_text) > 0:
    print("Exemples de textes 'spammy':")
    print(spammy_text.head(5).to_markdown())

# Auteurs suspects (identifiés comme 'power_users' précédemment)
print(f"\nNombre d'auteurs 'suspects' (> 50 posts): {len(power_users)}")

# Texte tout en majuscules
df['all_caps'] = df['text'].str.isupper() & (df['text_length'] > 10) # Ignorer les courts
all_caps_count = df['all_caps'].sum()
print(f"\nNombre de posts en majuscules (potentiel 'screaming'): {all_caps_count}")

 ## 9.  Rapport Final et Recommandations

 Synthèse des observations et des actions recommandées pour le pré-traitement.



 ### Executive Summary (Modèle)

 L'analyse exploratoire a porté sur [Total Posts] posts provenant de Reddit et Twitter, couvrant la période du [Date Min] au [Date Max]. L'activité est dominée par [Source Majoritaire] ([X.X]%). L'engagement montre une distribution très asymétrique, suggérant que quelques posts génèrent la majorité des interactions. L'analyse temporelle révèle des pics d'activité notables autour de [Date des Pics], indiquant potentiellement des événements majeurs du marché. L'analyse de texte montre que les discussions se concentrent sur [Top Mot 1] et [Top Mot 2], avec une forte prévalence de discussions autour de [Top Crypto]. Plusieurs problèmes de qualité des données ont été identifiés, notamment [X.X]% de doublons et une proportion significative de messages potentiellement "spammy".

 ###  Key Findings (Constatations Clés)

 * **Volume et Source:** Le dataset contient [Total Posts] posts, avec une répartition [X]% Twitter et [Y]% Reddit.
 * **Activité Temporelle:** L'activité est la plus forte le [Jour Max] et la plus faible le [Jour Min]. L'heure de pointe se situe autour de [Heure Max] (UTC/Local?).
 * **Tendance:** La tendance générale des posts est [à la hausse / à la baisse / stable] sur la période analysée.
 * **Contenu:** Les posts sont en moyenne de [Longueur Moyenne] caractères. [X]% des posts contiennent des URLs et [Y]% contiennent des emojis (le 🚀 est le plus populaire).
 * **Sujets:** [Top Crypto 1] et [Top Crypto 2] sont les cryptomonnaies les plus discutées. [X.X]% des posts mentionnent au moins une crypto de notre liste.
 * **Engagement:** L'engagement médian est de [Médiane Engagement], mais la moyenne est de [Moyenne Engagement], indiquant une forte asymétrie (skewness). [Source] génère en moyenne un engagement plus élevé (ou plus variable).
 * **Qualité:** [X]% des posts sont des doublons exacts. [Y] auteurs sont des "power users" (ou spammeurs) avec plus de 50 posts chacun.

 ###  Data Quality Issues (Problèmes de Qualité)

 1.  **Doublons:** [Nombre de Doublons] posts sont des doublons exacts et devraient être supprimés.
 2.  **Spam de Texte:** [Nombre de Textes Répétés] textes uniques sont répétés plus de 5 fois, indiquant un spam de type "copier-coller".
 3.  **Spam d'Auteurs:** [Nombre de Power Users] auteurs postent de manière excessive et devraient être examinés (pourraient être des bots ou des comptes de news légitimes).
 4.  **Posts Courts/Vides:** [Nombre de Posts Courts] posts ont moins de 10 caractères et n'apportent probablement aucune valeur (par ex. "ok", "lol").
 5.  **Multilingue:** Environ [X]% des posts ne sont pas en anglais (basé sur l'échantillon).

 ###  Recommendations for Cleaning (Recommandations pour le Nettoyage)

 1.  **Supprimer les doublons:** `df.drop_duplicates(subset=['text'], keep='first', inplace=True)`
 2.  **Filtrer les posts courts:** `df = df[df['text_length'] >= 10]`
 3.  **Filtrer les langues (si nécessaire):** Appliquer la détection de langue à l'ensemble du dataset (peut être lent) et conserver uniquement l'anglais (`df = df[df['language'] == 'en']`).
 4.  **Gérer le spam:**
     * **Texte:** Identifier les textes répétés > 5 fois et n'en garder qu'une seule instance (ou les marquer).
     * **Auteurs:** Envisager de plafonner le nombre de posts par auteur ou de les exclure si l'analyse se concentre sur l'opinion publique générale plutôt que sur les "influenceurs".
 5.  **Normalisation du texte:** Appliquer un pipeline de nettoyage NLP standard (minuscules, suppression des URLs, suppression de la ponctuation, suppression des emojis) avant la modélisation.
 6.  **Enrichissement:** Remplir la colonne `crypto_mentioned` en utilisant une liste de regex plus exhaustive.